# pandas pivot tables

In [1]:
import pandas as pd

In [2]:
melb_df = pd.read_csv("data/melb_data_fe.csv", index_col=False)
melb_df.head(3)

,Suburb,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,Bedroom,...,Longtitude,Regionname,Propertycount,MeanRoomsSquare,AreaRatio,MonthSale,AgeBuilding,WeekdaySale,StreetType,Weekend
0,Abbotsford,2,house,1480000.0,S,Biggin,2016-03-12,2.5,3067,2,...,144.9984,Northern Metropolitan,4019,25.20,-0.231707,3,46,5,St,1
1,Abbotsford,2,house,1035000.0,S,Biggin,2016-04-02,2.5,3067,2,...,144.9934,Northern Metropolitan,4019,15.80,-0.327660,4,116,5,St,1
2,Abbotsford,3,house,1465000.0,SP,Biggin,2017-04-03,2.5,3067,3,...,144.9944,Northern Metropolitan,4019,18.75,0.056338,4,117,0,St,0


### .groupby() as way to creating a pivot table

In [3]:
melb_df.groupby("Rooms")[["Price", "BuildingArea"]].median()

,Price,BuildingArea
Rooms,,
1,385000.0,107.0
2,690000.0,126.0
3,950000.0,126.0
4,1285000.0,142.0
5,1660000.0,176.0
6,1800000.0,126.0
7,1496000.0,216.5
8,1515000.0,126.0
10,900000.0,126.0


In [4]:
melb_df.groupby(["Rooms", "Type"])["Price"].mean()

Rooms  Type     
1      house        8.668655e+05
       townhouse    5.927045e+05
       unit         3.899289e+05
2      house        1.017238e+06
       townhouse    7.101585e+05
       unit         6.104905e+05
3      house        1.109233e+06
       townhouse    9.847087e+05
       unit         8.505963e+05
4      house        1.462283e+06
       townhouse    1.217092e+06
       unit         1.037476e+06
5      house        1.877327e+06
       townhouse    1.035000e+06
6      house        1.869508e+06
       unit         5.200000e+05
7      house        1.920700e+06
8      house        1.510286e+06
       unit         2.250000e+06
10     house        9.000000e+05
Name: Price, dtype: float64

In [5]:
melb_df.groupby(["Rooms", "Type"])["Price"].mean().unstack()

Type,house,townhouse,unit
Rooms,,,
1,8.668655e+05,5.927045e+05,3.899289e+05
2,1.017238e+06,7.101585e+05,6.104905e+05
3,1.109233e+06,9.847087e+05,8.505963e+05
4,1.462283e+06,1.217092e+06,1.037476e+06
5,1.877327e+06,1.035000e+06,NaN
6,1.869508e+06,NaN,5.200000e+05
7,1.920700e+06,NaN,NaN
8,1.510286e+06,NaN,2.250000e+06
10,9.000000e+05,NaN,NaN


### .pivot_table()
* values= name of column, that needs to aggregate
* index= name of column, whose values are rows of pivot table
* columns= name of column, whose values are column of pivot table
* aggfunc= name or name list of aggregating functions 'mean' as default
* fill_value= item for empty value as default empty value stay empty

In [12]:
melb_df.pivot_table(
    values='Price',
    index='Rooms',
    columns='Type',
    fill_value=0
).round()

Type,house,townhouse,unit
Rooms,,,
1,866866.0,592705.0,389929.0
2,1017238.0,710158.0,610491.0
3,1109233.0,984709.0,850596.0
4,1462283.0,1217092.0,1037476.0
5,1877327.0,1035000.0,0.0
6,1869508.0,0.0,520000.0
7,1920700.0,0.0,0.0
8,1510286.0,0.0,2250000.0
10,900000.0,0.0,0.0


In [15]:
melb_df.pivot_table(
    values='Price',
    index='Regionname',
    columns='Weekend',
    aggfunc='count'
)

Weekend,0,1
Regionname,,
Eastern Metropolitan,447,1024
Eastern Victoria,13,40
Northern Metropolitan,1258,2632
Northern Victoria,11,30
South-Eastern Metropolitan,123,327
Southern Metropolitan,1534,3161
Western Metropolitan,960,1988
Western Victoria,8,24


In [17]:
melb_df.pivot_table(
    values='Landsize',
    index='Regionname',
    columns='Type',
    aggfunc=['median', 'mean'],
    fill_value=0
)

median                          mean              \
Type                        house townhouse   unit        house   townhouse   
Regionname                                                                    
Eastern Metropolitan        674.0     233.5  203.0   717.422847  269.440678   
Eastern Victoria            843.0       0.0  230.0  3108.960000    0.000000   
Northern Metropolitan       459.5     134.0    0.0   619.249092  317.325733   
Northern Victoria           724.0       0.0    0.0  3355.463415    0.000000   
South-Eastern Metropolitan  630.5     240.0  199.0   664.306701  212.160000   
Southern Metropolitan       586.0     246.0    0.0   569.643881  278.858824   
Western Metropolitan        531.0     198.0   62.0   507.883406  244.560669   
Western Victoria            599.5       0.0    0.0   655.500000    0.000000   

                                        
Type                              unit  
Regionname                              
Eastern Metropolitan        330.444444  
Eastern Victoria            295.333333  
Northern Metropolitan       495.026538  
Northern Victoria             0.000000  
South-Eastern Metropolitan  357.864865  
Southern Metropolitan       466.380245  
Western Metropolitan        557.637232  
Western Victoria              0.000000

### multidimensional pivot table

In [18]:
melb_df.pivot_table(
    values='Price',
    index=['Method', 'Type'],
    columns='Regionname',
    aggfunc='median',
    fill_value=0
)

Regionname        Eastern Metropolitan  Eastern Victoria  \
Method Type                                                
PI     house                 1244000.0          780000.0   
       townhouse              760000.0               0.0   
       unit                   650000.0               0.0   
S      house                 1127000.0          675000.0   
       townhouse              828000.0               0.0   
       unit                   645750.0          492000.0   
SA     house                  932500.0          950000.0   
       townhouse              807500.0               0.0   
       unit                        0.0               0.0   
SP     house                 1050000.0          672500.0   
       townhouse              910000.0               0.0   
       unit                   515000.0          400000.0   
VB     house                 1100000.0          712500.0   
       townhouse              892500.0               0.0   
       unit                   500000.0               0.0   

Regionname        Northern Metropolitan  Northern Victoria  \
Method Type                                                  
PI     house                   900000.0           500000.0   
       townhouse               632500.0                0.0   
       unit                    410000.0                0.0   
S      house                   920000.0           555000.0   
       townhouse               750000.0                0.0   
       unit                    525500.0                0.0   
SA     house                   817500.0           540000.0   
       townhouse               425000.0                0.0   
       unit                    616000.0                0.0   
SP     house                   900000.0           521000.0   
       townhouse               690000.0                0.0   
       unit                    470000.0                0.0   
VB     house                  1050000.0           690000.0   
       townhouse               640000.0                0.0   
       unit                    450000.0                0.0   

Regionname        South-Eastern Metropolitan  Southern Metropolitan  \
Method Type                                                           
PI     house                        865000.0              1725000.0   
       townhouse                   1190000.0              1055000.0   
       unit                         525000.0               571250.0   
S      house                        883300.0              1611000.0   
       townhouse                    875000.0              1135000.0   
       unit                         606000.0               655000.0   
SA     house                        880000.0              1390000.0   
       townhouse                         0.0              1141000.0   
       unit                              0.0               580000.0   
SP     house                        770000.0              1521750.0   
       townhouse                    800000.0              1162500.0   
       unit                         601000.0               550000.0   
VB     house                        850000.0              1800000.0   
       townhouse                         0.0              1250000.0   
       unit                         700000.0               500000.0   

Regionname        Western Metropolitan  Western Victoria  
Method Type                                               
PI     house                  870000.0          630000.0  
       townhouse              670000.0               0.0  
       unit                   360000.0               0.0  
S      house                  870000.0          397500.0  
       townhouse              729000.0               0.0  
       unit                   489000.0               0.0  
SA     house                  772500.0               0.0  
       townhouse              467500.0               0.0  
       unit                   571000.0               0.0  
SP     house                  865000.0          360000.0  
       t

### access to data of pivot table

In [19]:
pivot = melb_df.pivot_table(
    values='Landsize',
    index='Regionname',
    columns='Type',
    aggfunc=['median', 'mean'],
    fill_value=0
)

In [20]:
pivot.columns

MultiIndex([('median',     'house'),
            ('median', 'townhouse'),
            ('median',      'unit'),
            (  'mean',     'house'),
            (  'mean', 'townhouse'),
            (  'mean',      'unit')],
           names=[None, 'Type'])

In [23]:
print(type(pivot['mean']['house']))
pivot['mean']['house']

<class 'pandas.core.series.Series'>


Regionname
Eastern Metropolitan           717.422847
Eastern Victoria              3108.960000
Northern Metropolitan          619.249092
Northern Victoria             3355.463415
South-Eastern Metropolitan     664.306701
Southern Metropolitan          569.643881
Western Metropolitan           507.883406
Western Victoria               655.500000
Name: house, dtype: float64

### filtering values of pivit table

In [24]:
mask = pivot['mean']['house'] < pivot['median']['house']
pivot[mask]

median                        mean              \
Type                   house townhouse  unit       house   townhouse   
Regionname                                                             
Southern Metropolitan  586.0     246.0   0.0  569.643881  278.858824   
Western Metropolitan   531.0     198.0  62.0  507.883406  244.560669   

                                   
Type                         unit  
Regionname                         
Southern Metropolitan  466.380245  
Western Metropolitan   557.637232

### to get indexes of filtered table 

In [27]:
list_pivot_index = list(pivot[mask].index)
print(list_pivot_index)

['Southern Metropolitan', 'Western Metropolitan']


### to create multidemensial table

In [35]:
import numpy as np
mseries = pd.Series(
    np.random.randint(0,10, size=8),
    index =[['white','white','white','blue','blue','red','red','red'],
            ['up','down','right','up','down','up','down','left']]       
)
display(mseries)
print(mseries.index)

white  up       6
       down     6
       right    6
blue   up       7
       down     9
red    up       1
       down     3
       left     5
dtype: int32

MultiIndex([('white',    'up'),
            ('white',  'down'),
            ('white', 'right'),
            ( 'blue',    'up'),
            ( 'blue',  'down'),
            (  'red',    'up'),
            (  'red',  'down'),
            (  'red',  'left')],
           )


In [36]:
mframe = pd.DataFrame(
    np.random.randn(16).reshape(4,4),
    index=[['white','white','red','red'], ['up','down','up','down']],
    columns=[['pen','pen','paper','paper'],[1,2,1,2]]
)
display(mframe)

pen               paper          
                   1         2         1         2
white up   -0.474740 -0.218389  0.438283  1.079320
      down  0.235553 -1.189413  0.857899 -0.207781
red   up    1.136114  0.562132 -1.199149  1.524466
      down  1.038290  2.676018 -1.471519  0.413002

In [39]:
melb_df.pivot_table(
    values='BuildingArea',
    index='Rooms',
    columns='Type',
    aggfunc='median'
)

Type,house,townhouse,unit
Rooms,,,
1,126.0,88.0,69.5
2,126.0,114.0,110.0
3,126.0,126.0,126.0
4,141.0,159.5,126.0
5,177.0,152.0,NaN
6,126.0,NaN,171.0
7,216.5,NaN,NaN
8,126.0,NaN,126.0
10,126.0,NaN,NaN


In [43]:
melb_df.pivot_table(
    values='BuildingArea',
    index='Type',
    columns='Rooms',
    aggfunc='median'
)

Rooms,1,2,3,4,5,6,7,8,10
Type,,,,,,,,,
house,126.0,126.0,126.0,141.0,177.0,126.0,216.5,126.0,126.0
townhouse,88.0,114.0,126.0,159.5,152.0,NaN,NaN,NaN,NaN
unit,69.5,110.0,126.0,126.0,NaN,171.0,NaN,126.0,NaN


In [72]:
seller_price = melb_df.pivot_table(
    values='Price',
    index='SellerG',
    columns='Type',
    aggfunc='median'
)
mask1 = seller_price['unit'] == seller_price['unit'].max()
display(seller_price.index[mask1].to_list())

['Nick']